In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import datetime
import os
import sys
import getopt
import errno

sys.path.insert(0,"/home/sunnycui/deepcell-tf")

MODEL_DIR = os.path.join(sys.path[0], 'scripts/recurr_gru/models/')
LOG_DIR = os.path.join(sys.path[0], 'scripts/recurr_gru/logs/')
DATA_DIR = os.path.join(sys.path[0], 'scripts/recurr_gru/data/')

In [2]:
import math
import numpy as np
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.optimizers import SGD

from tensorflow.python.keras.utils.data_utils import get_file

import deepcell
from deepcell import losses
from scripts.recurr_gru import image_gen
from deepcell import image_generators
from deepcell import model_zoo

from deepcell.utils import train_utils
from deepcell.utils.data_utils import get_data
from deepcell.utils.train_utils import rate_scheduler
from deepcell.training import train_model_conv

from sklearn.model_selection import train_test_split

from tensorflow.python.client import device_lib

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
import math
import numpy as np
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.optimizers import SGD

from tensorflow.python.keras.utils.data_utils import get_file

import deepcell
from deepcell import losses
from scripts.recurr_gru import image_gen
from deepcell import image_generators
from deepcell import model_zoo
from deepcell.layers import TensorProduct, ReflectionPadding3D, DilatedMaxPool3D

from deepcell.utils import train_utils
from deepcell.utils.data_utils import get_data
from deepcell.utils.train_utils import rate_scheduler
from deepcell.training import train_model_conv


from tensorflow.python.keras.layers import MaxPool3D, Conv3DTranspose, UpSampling3D
from scripts.recurr_gru.conv_gru_layer import ConvGRU2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.layers import Conv3D, ZeroPadding3D, ConvLSTM2D, Cropping3D
from tensorflow.python.keras.layers import Input, Add, Concatenate, Flatten
from tensorflow.python.keras.engine.input_layer import InputLayer

from tensorflow.python.keras.models import Model


from tensorflow.python.keras.regularizers import l2
from deepcell.layers import ImageNormalization3D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Softmax

from sklearn.model_selection import train_test_split

from tensorflow.python.client import device_lib

# Load Data

In [4]:
filename = 'combined_is_nuclei.npz'
DATA_FILE = os.path.join(DATA_DIR, filename)

print("Loading data from " + filename)
train_dict, test_dict = get_data(DATA_FILE, mode='conv', test_size=0.1, seed=0)

print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))

Loading data from combined_is_nuclei.npz


FileNotFoundError: [Errno 2] No such file or directory: '/home/sunnycui/deepcell-tf/scripts/recurr_gru/data/combined_is_nuclei.npz'

### Training params

In [ ]:
fgbg_model_name = 'nuclei_model_fgbg'
watershed_model_name = 'nuclei_model_watershed'


n_epoch = 20  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = None  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 0  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

# Transformation settings
transform = 'watershed'
dilation_radius = 1  # change dilation radius for edge dilation

# 3D Settings
frames_per_batch = 3
# norm_method = None #'whole_image'

distance_bins = 4  # number of distance classes
erosion_width = 0  # erode edges

In [147]:
def feature_net_3D(receptive_field=61,
                    n_frames=3,
                    input_shape=(5, 256, 256, 1),
                    n_features=3,
                    n_channels=1,
                    reg=1e-5,
                    n_conv_filters=64,
                    n_dense_filters=200,
                    gru_kernel_size =3,
                    VGG_mode=False,
                    init='he_normal',
                    norm_method='whole_image',
                    gru=False,
                    location=False,
                    dilated=False,
                    padding=False,
                    padding_mode='reflect',
                    multires=False,
                    include_top=True):
    # Create layers list (x) to store all of the layers.
    # We need to use the functional API to enable the multiresolution mode
    x = []

    win = (receptive_field - 1) // 2
    win_z = (n_frames - 1) // 2

    if dilated:
        padding = True

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        time_axis = 2
        row_axis = 3
        col_axis = 4
        if not dilated:
            input_shape = (n_channels, n_frames, receptive_field, receptive_field)
    else:
        channel_axis = -1
        time_axis = 1
        row_axis = 2
        col_axis = 3
        if not dilated:
            input_shape = (n_frames, receptive_field, receptive_field, n_channels)

    x.append(Input(shape=input_shape))
    # x.append(ImageNormalization3D(norm_method=norm_method, filter_size=receptive_field)(x[-1]))
    # x.append(BatchNormalization(axis=channel_axis)(x[-1]))

    if padding:
        if padding_mode == 'reflect':
            x.append(ReflectionPadding3D(padding=(win_z, win, win))(x[-1]))
        elif padding_mode == 'zero':
            x.append(ZeroPadding3D(padding=(win_z, win, win))([-1]))

    if location:
        x.append(Location3D(in_shape=tuple(x[-1].shape.as_list()[1:]))(x[-1]))
        x.append(Concatenate(axis=channel_axis)([x[-2], x[-1]]))

    if multires:
        layers_to_concat = []

    rf_counter = receptive_field
    block_counter = 0
    d = 1

    append_gru = False
    while rf_counter > 4:
        filter_size = 3 if rf_counter % 2 == 0 else 4
        if append_gru == True:
            x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(filter_size, filter_size), dilation_rate=(d, d),
                            padding='valid', kernel_initializer=init,
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        else:
            x.append(Conv3D(n_conv_filters, (1, filter_size, filter_size), 
                            dilation_rate=(1, d, d), kernel_initializer=init,
                            padding='valid', kernel_regularizer=l2(reg))(x[-1]))

        
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))

        block_counter += 1
        rf_counter -= filter_size - 1

        if block_counter % 2 == 0:
            if dilated:
                x.append(DilatedMaxPool3D(dilation_rate=(1, d, d), pool_size=(1, 2, 2))(x[-1]))
                d *= 2
            else:
                x.append(MaxPool3D(pool_size=(1, 2, 2))(x[-1]))

            if VGG_mode:
                n_conv_filters *= 2

            rf_counter = rf_counter // 2

            if multires:
                layers_to_concat.append(len(x) - 1)

    if multires:
        c = []
        for l in layers_to_concat:
            output_shape = x[l].get_shape().as_list()
            target_shape = x[-1].get_shape().as_list()
            time_crop = (0, 0)

            row_crop = int(output_shape[row_axis] - target_shape[row_axis])

            if row_crop % 2 == 0:
                row_crop = (row_crop // 2, row_crop // 2)
            else:
                row_crop = (row_crop // 2, row_crop // 2 + 1)

            col_crop = int(output_shape[col_axis] - target_shape[col_axis])

            if col_crop % 2 == 0:
                col_crop = (col_crop // 2, col_crop // 2)
            else:
                col_crop = (col_crop // 2, col_crop // 2 + 1)

            cropping = (time_crop, row_crop, col_crop)

            c.append(Cropping3D(cropping=cropping)(x[l]))
        x.append(Concatenate(axis=channel_axis)(c))
        
    
    x.append(Conv3D(n_dense_filters, (1, rf_counter, rf_counter), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(Conv3D(n_dense_filters, (n_frames, 1, 1), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))
    
    if gru == True:
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size, gru_kernel_size),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        # x.append(Activation('relu')(x[-1]))
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size +2, gru_kernel_size +2),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        # x.append(Activation('relu')(x[-1]))
    
    
    x.append(TensorProduct(n_dense_filters, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(TensorProduct(n_features, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))

    if not dilated:
        x.append(Flatten()(x[-1]))

    if include_top:
        x.append(Softmax(axis=channel_axis)(x[-1]))

    model = Model(inputs=x[0], outputs=x[-1])
    model.summary()

    return model

In [148]:
from tensorflow.python.keras.layers import Lambda;

def image_norm(inputs):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    axes = [3, 4] if channel_axis == 1 else [2, 3]
    output = inputs - K.mean(inputs, axis=axes, keepdims=True)
    output = output / K.std(inputs, axis=axes, keepdims=True)
    return output

def feature_net_skip_3D(receptive_field=61,
                        input_shape=(5, 256, 256, 1),
                        fgbg_model=None,
                        gru=False,
                        gru_kernel_size=3,
                        last_only=True,
                        n_skips=1,
                        norm_method='whole_image',
                        padding_mode='reflect',
                        **kwargs):
    # print(K.image_data_format())
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    inputs = Input(shape=input_shape)
    
    img = Lambda(image_norm)(inputs)
    #img = BatchNormalization(axis=channel_axis)(inputs)

    models = []
    model_outputs = []

    if fgbg_model is not None:
        for layer in fgbg_model.layers:
            layer.trainable = False
        models.append(fgbg_model)
        fgbg_output = fgbg_model(inputs)
        if isinstance(fgbg_output, list):
            fgbg_output = fgbg_output[-1]
        model_outputs.append(fgbg_output)

    for _ in range(n_skips + 1):
        if model_outputs:
            model_input = Concatenate(axis=channel_axis)([img, model_outputs[-1]])
        else:
            model_input = img
        new_input_shape = model_input.get_shape().as_list()[1:]
        models.append(feature_net_3D(receptive_field=receptive_field, 
                                     input_shape=new_input_shape, norm_method=None, dilated=True, 
                                     padding=True, padding_mode=padding_mode, gru=gru, 
                                     gru_kernel_size=gru_kernel_size, **kwargs))
        model_outputs.append(models[-1](model_input))

    if last_only:
        model = Model(inputs=inputs, outputs=model_outputs[-1])
    else:
        if fgbg_model is None:
            model = Model(inputs=inputs, outputs=model_outputs)
        else:
            model = Model(inputs=inputs, outputs=model_outputs[1:])
    model.summary()
    return model

# Train

In [149]:
fgbg_model = feature_net_skip_3D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    n_frames=frames_per_batch,
    gru=False,
    gru_kernel_size=3,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=tuple([frames_per_batch] + list(train_dict['X'].shape[2:])),
    last_only=False)

watershed_model = feature_net_skip_3D(
    n_features=distance_bins,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    n_frames=frames_per_batch,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=tuple([frames_per_batch] + list(train_dict['X'].shape[2:])),
    last_only=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_74 (InputLayer)        (None, 3, 160, 160, 1)    0         
_________________________________________________________________
reflection_padding3d_36 (Ref (None, 5, 220, 220, 1)    0         
_________________________________________________________________
conv3d_292 (Conv3D)          (None, 5, 217, 217, 32)   544       
_________________________________________________________________
batch_normalization_328 (Bat (None, 5, 217, 217, 32)   128       
_________________________________________________________________
activation_328 (Activation)  (None, 5, 217, 217, 32)   0         
_________________________________________________________________
conv3d_293 (Conv3D)          (None, 5, 215, 215, 32)   9248      
_________________________________________________________________
batch_normalization_329 (Bat (None, 5, 215, 215, 32)   128       
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_75 (InputLayer)        (None, 3, 160, 160, 1)    0         
_________________________________________________________________
lambda_37 (Lambda)           (None, 3, 160, 160, 1)    0         
_________________________________________________________________
model_74 (Model)             (None, 3, 160, 160, 4)    181060    
Total params: 181,060
Trainable params: 179,908
Non-trainable params: 1,152
_________________________________________________________________


In [151]:
from deepcell.training import train_model_conv

fgbg_model = train_model_conv(
    model=fgbg_model,
    train_dict=train_dict,
    test_dict=test_dict,
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    n_epoch=n_epoch,
    batch_size=batch_size,
    frames_per_batch=frames_per_batch,
    transform='fgbg',
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False)

watershed_model = train_model_conv(
    model=watershed_model,
    train_dict=train_dict,
    test_dict=test_dict,
    model_name=watershed_model_name,
    test_size=test_size,
    optimizer=optimizer,
    n_epoch=n_epoch,
    batch_size=batch_size,
    frames_per_batch=frames_per_batch,
    transform='watershed',
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False)

X_train shape: (8, 24, 160, 160, 1)
y_train shape: (8, 24, 160, 160, 1)
X_test shape: (1, 24, 160, 160, 1)
y_test shape: (1, 24, 160, 160, 1)
Output Shape: (None, 3, 160, 160, 2)
Number of Classes: 2
Training on 1 GPUs
Epoch 1/20
7/8 [=========================>....] - ETA: 2s - loss: 1.0675 - acc: 0.7545
Epoch 00001: val_loss improved from inf to 7.16207, saving model to /home/sunnycui/deepcell-tf/scripts/recurr_gru/models/nuclei_model_fgbg.h5
8/8 [==============================] - 26s 3s/step - loss: 1.0280 - acc: 0.7487 - val_loss: 7.1621 - val_acc: 0.1297
Epoch 2/20
7/8 [=========================>....] - ETA: 0s - loss: 0.6125 - acc: 0.8267
Epoch 00002: val_loss improved from 7.16207 to 4.92512, saving model to /home/sunnycui/deepcell-tf/scripts/recurr_gru/models/nuclei_model_fgbg.h5
8/8 [==============================] - 1s 93ms/step - loss: 0.5652 - acc: 0.8405 - val_loss: 4.9251 - val_acc: 0.1005
Epoch 3/20
7/8 [=========================>....] - ETA: 0s - loss: 0.3783 - acc: 0.92

8/8 [==============================] - 1s 89ms/step - loss: 0.7986 - acc: 0.8617 - val_loss: 3.9124 - val_acc: 0.6250
Epoch 6/20
7/8 [=========================>....] - ETA: 0s - loss: 0.9640 - acc: 0.8865
Epoch 00006: val_loss did not improve from 3.91242
8/8 [==============================] - 1s 81ms/step - loss: 0.9683 - acc: 0.8977 - val_loss: 7.4156 - val_acc: 0.7175
Epoch 7/20
7/8 [=========================>....] - ETA: 0s - loss: 1.0831 - acc: 0.8809
Epoch 00007: val_loss improved from 3.91242 to 1.79661, saving model to /home/sunnycui/deepcell-tf/scripts/recurr_gru/models/nuclei_model_watershed.h5
8/8 [==============================] - 1s 90ms/step - loss: 1.0300 - acc: 0.8835 - val_loss: 1.7966 - val_acc: 0.5803
Epoch 8/20
7/8 [=========================>....] - ETA: 0s - loss: 0.8966 - acc: 0.8831
Epoch 00008: val_loss did not improve from 1.79661
8/8 [==============================] - 1s 81ms/step - loss: 0.8952 - acc: 0.8797 - val_loss: 2.3594 - val_acc: 0.4651
Epoch 9/20
7/8

In [152]:
fgbg_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_model_name))
fgbg_model.save_weights(fgbg_weights_file)

watershed_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(watershed_model_name))
watershed_model.save_weights(watershed_weights_file)

# Test Images

In [153]:

run_fgbg_model = feature_net_skip_3D(
    receptive_field=receptive_field,
    n_features=2,
    n_frames=frames_per_batch,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=tuple(test_dict['X'].shape[1:]),
    multires=False,
    last_only=False,
    norm_method=norm_method)
run_fgbg_model.load_weights(fgbg_weights_file)

run_watershed_model = feature_net_skip_3D(
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_features=distance_bins,
    n_frames=frames_per_batch,
    gru=False,
    gru_kernel_size=3,
    n_conv_filters=32,
    n_dense_filters=128,
    multires=False,
    last_only=False,
    input_shape=tuple(test_dict['X'].shape[1:]),
    norm_method=norm_method)

run_watershed_model.load_weights(watershed_weights_file)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_78 (InputLayer)        (None, 24, 160, 160, 1)   0         
_________________________________________________________________
reflection_padding3d_38 (Ref (None, 26, 220, 220, 1)   0         
_________________________________________________________________
conv3d_308 (Conv3D)          (None, 26, 217, 217, 32)  544       
_________________________________________________________________
batch_normalization_346 (Bat (None, 26, 217, 217, 32)  128       
_________________________________________________________________
activation_346 (Activation)  (None, 26, 217, 217, 32)  0         
_________________________________________________________________
conv3d_309 (Conv3D)          (None, 26, 215, 215, 32)  9248      
_________________________________________________________________
batch_normalization_347 (Bat (None, 26, 215, 215, 32)  128       
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_79 (InputLayer)        (None, 24, 160, 160, 1)   0         
_________________________________________________________________
lambda_39 (Lambda)           (None, 24, 160, 160, 1)   0         
_________________________________________________________________
model_78 (Model)             (None, 24, 160, 160, 4)   181060    
Total params: 181,060
Trainable params: 179,908
Non-trainable params: 1,152
_________________________________________________________________


In [154]:
test_images = run_watershed_model.predict(test_dict['X'])[-1]
test_images_fgbg = run_fgbg_model.predict(test_dict['X'])[-1]

# test_images = watershed_model.predict(test_dict['X'])[-1]
# test_images_fgbg = fgbg_model.predict(test_dict['X'])[-1]

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

watershed transform shape: (24, 160, 160, 4)
segmentation mask shape: (24, 160, 160, 2)


### Post process

In [155]:
argmax_images = []
for i in range(test_images.shape[0]):
    max_image = np.argmax(test_images[i], axis=-1)
    argmax_images.append(max_image)
argmax_images = np.array(argmax_images)
argmax_images = np.expand_dims(argmax_images, axis=-1)

print('watershed argmax shape:', argmax_images.shape)

watershed argmax shape: (24, 160, 160, 1)


In [156]:
# Apply watershed method with the distance transform as seed
from skimage.measure import label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

# threshold the foreground/background
# and remove back ground from watershed transform
threshold = 0.5

fg_thresh = test_images_fgbg[..., 1] > threshold
fg_thresh = np.expand_dims(fg_thresh, axis=-1)

argmax_images_post_fgbg = argmax_images * fg_thresh


watershed_images = []
for i in range(argmax_images_post_fgbg.shape[0]):
    image = fg_thresh[i, ..., 0]
    distance = argmax_images_post_fgbg[i, ..., 0]

    local_maxi = peak_local_max(
        test_images[i, ..., -1],
        min_distance=10,
        threshold_abs=0.05,
        indices=False,
        labels=image,
        exclude_border=False)

    markers = label(local_maxi)
    segments = watershed(-distance, markers, mask=image)
    watershed_images.append(segments)

watershed_images = np.array(watershed_images)
watershed_images = np.expand_dims(watershed_images, axis=-1)

In [157]:
import matplotlib.pyplot as plt

frame = np.random.randint(low=0, high=watershed_images.shape[0])

print('Frame:', frame)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(test_dict['X'][0, frame, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images[frame, ..., 1])
ax[1].set_title('FGBG Prediction')

ax[2].imshow(fg_thresh[frame, ..., 0], cmap='jet')
ax[2].set_title('FGBG {}% Threshold'.format(int(threshold * 100)))

ax[3].imshow(argmax_images[frame, ..., 0], cmap='jet')
ax[3].set_title('Distance Transform')

ax[4].imshow(argmax_images_post_fgbg[frame, ..., 0], cmap='jet')
ax[4].set_title('Distance Transform without background')

ax[5].imshow(watershed_images[frame, ..., 0], cmap='jet')
ax[5].set_title('Watershed Segmentation')

# ax[5].imshow(test_dict['y'][0, frame, ..., 0], cmap='jet')
# ax[5].set_title('Ground truth')

fig.tight_layout()
plt.show()
plt.savefig('watershed_nuclei')

Frame: 22


In [103]:
print(test_dict['y'].shape)

(1, 24, 160, 160, 1)


# Train for Lumen

X0 = DAPI   

X1 = GFP  

y0 = lumens  

y1 = organoid

In [97]:
from sklearn.model_selection import train_test_split


filename = 'combined_is_all.npz'
DATA_FILE = os.path.join(DATA_DIR, filename)

combined_npz = np.load(DATA_FILE)
X_full, y_full = combined_npz['X'], combined_npz['y']
print(' -\nX.shape: {}\ny.shape: {}'.format(X_full.shape, y_full.shape))

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.1, random_state=0)


train_dict = {
    'X': np.swapaxes(X_train, 0, 1)[1],
    'y': np.swapaxes(y_train, 0, 1)[1],
}


test_dict = {
    'X': np.swapaxes(X_test, 0, 1)[1],
    'y': np.swapaxes(y_test, 0, 1)[1],
}

print(' -\nX_train.shape: {}\ny_train.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))
print(' -\nX_test.shape: {}\ny_test.shape: {}'.format(test_dict['X'].shape, test_dict['y'].shape))

 -
X.shape: (222, 2, 160, 160, 1)
y.shape: (222, 2, 160, 160, 1)
 -
X_train.shape: (199, 160, 160, 1)
y_train.shape: (199, 160, 160, 1)
 -
X_test.shape: (23, 160, 160, 1)
y_test.shape: (23, 160, 160, 1)


In [106]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

fgbg_model_name = 'organoid_fgbg_model'
conv_model_name = 'organoid_watershed_model'

n_epoch = 15  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = 'std'  # data normalization
receptive_field = 81  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 0  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

# Transformation settings
transform = 'watershed'
distance_bins = 4
erosion_width = 0  # erode edges

In [107]:
from deepcell import model_zoo
from deepcell.training import train_model_conv


fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=tuple(train_dict['X'].shape[1:]),
    last_only=False)

fgbg_model = train_model_conv(
    model=fgbg_model,
    train_dict=train_dict,
    test_dict=test_dict,
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    n_epoch=n_epoch,
    batch_size=batch_size,
    transform='fgbg',
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2))

X_train shape: (199, 160, 160, 1)
y_train shape: (199, 160, 160, 1)
X_test shape: (23, 160, 160, 1)
y_test shape: (23, 160, 160, 1)
Output Shape: (None, 160, 160, 2)
Number of Classes: 2
Training on 1 GPUs
Epoch 1/15
197/199 [============================>.] - ETA: 0s - loss: 0.3250 - acc: 0.8109
Epoch 00001: val_loss improved from inf to 0.22732, saving model to /home/sunnycui/deepcell-tf/scripts/recurr_gru/models/organoid_fgbg_model.h5
199/199 [==============================] - 22s 110ms/step - loss: 0.3239 - acc: 0.8115 - val_loss: 0.2273 - val_acc: 0.8823
Epoch 2/15
197/199 [============================>.] - ETA: 0s - loss: 0.2640 - acc: 0.8520
Epoch 00002: val_loss improved from 0.22732 to 0.21487, saving model to /home/sunnycui/deepcell-tf/scripts/recurr_gru/models/organoid_fgbg_model.h5
199/199 [==============================] - 6s 28ms/step - loss: 0.2632 - acc: 0.8528 - val_loss: 0.2149 - val_acc: 0.8840
Epoch 3/15
197/199 [============================>.] - ETA: 0s - loss: 0.23

In [108]:
from deepcell import model_zoo
from deepcell.training import train_model_conv


watershed_model = model_zoo.bn_feature_net_skip_2D(
    fgbg_model=fgbg_model,
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_features=distance_bins,
    n_conv_filters=32,
    n_dense_filters=128,
    last_only=False,
    input_shape=tuple(train_dict['X'].shape[1:]))

watershed_model = train_model_conv(
    model=watershed_model,
    train_dict=train_dict,
    test_dict=test_dict,
    model_name=conv_model_name,
    test_size=test_size,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=n_epoch,
    transform=transform,
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2))

X_train shape: (199, 160, 160, 1)
y_train shape: (199, 160, 160, 1)
X_test shape: (23, 160, 160, 1)
y_test shape: (23, 160, 160, 1)
Output Shape: (None, 160, 160, 4)
Number of Classes: 4
Training on 1 GPUs
Epoch 1/15
197/199 [============================>.] - ETA: 0s - loss: 1.1685 - acc: 0.5948
Epoch 00001: val_loss improved from inf to 1.37706, saving model to /home/sunnycui/deepcell-tf/scripts/recurr_gru/models/organoid_watershed_model.h5
199/199 [==============================] - 25s 124ms/step - loss: 1.1650 - acc: 0.5964 - val_loss: 1.3771 - val_acc: 0.6565
Epoch 2/15
197/199 [============================>.] - ETA: 0s - loss: 0.9821 - acc: 0.6564
Epoch 00002: val_loss did not improve from 1.37706
199/199 [==============================] - 8s 40ms/step - loss: 0.9795 - acc: 0.6577 - val_loss: 1.4026 - val_acc: 0.7247
Epoch 3/15
197/199 [============================>.] - ETA: 0s - loss: 0.9702 - acc: 0.6758
Epoch 00003: val_loss did not improve from 1.37706
199/199 [===============

In [109]:
# make predictions on testing data
test_images = watershed_model.predict(test_dict['X'])
test_images_fgbg = fgbg_model.predict(test_dict['X'])

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

# Collapse predictions into semantic segmentation mask

argmax_images = []
for i in range(test_images.shape[0]):
    max_image = np.argmax(test_images[i], axis=-1)
    argmax_images.append(max_image)
argmax_images = np.array(argmax_images)
argmax_images = np.expand_dims(argmax_images, axis=-1)

print('watershed argmax shape:', argmax_images.shape)

# threshold the foreground/background
# and remove background from watershed transform
threshold = 0.5
fg_thresh = test_images_fgbg[..., 1] > threshold

fg_thresh = np.expand_dims(fg_thresh.astype('int16'), axis=-1)
argmax_images_post_fgbg = argmax_images * fg_thresh

watershed transform shape: (23, 160, 160, 4)
segmentation mask shape: (23, 160, 160, 2)
watershed argmax shape: (23, 160, 160, 1)


In [110]:
# Apply watershed method with the distance transform as seed
from skimage.measure import label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

watershed_images = []
for i in range(argmax_images_post_fgbg.shape[0]):
    image = fg_thresh[i, ..., 0]
    distance = argmax_images_post_fgbg[i, ..., 0]

    local_maxi = peak_local_max(test_images[i, ..., -1], min_distance=15, 
                                exclude_border=False, indices=False, labels=image)

    markers = label(local_maxi)
    segments = watershed(-distance, markers, mask=image)
    watershed_images.append(segments)

watershed_images = np.array(watershed_images)
watershed_images = np.expand_dims(watershed_images, axis=-1)

In [125]:
import matplotlib.pyplot as plt

index = np.random.randint(low=0, high=test_dict['X'].shape[0])
print('Image number:', index)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(test_dict['X'][index, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, ..., 1])
ax[1].set_title('Organoid Prediction')

ax[2].imshow(fg_thresh[index, ..., 0], cmap='jet')
ax[2].set_title('FGBG Threshold {}%'.format(threshold * 100))

ax[3].imshow(test_dict['y'][index, ..., 0], cmap='jet')
ax[3].set_title('Ground truth')

ax[4].imshow(argmax_images_post_fgbg[index, ..., 0], cmap='jet')
ax[4].set_title('Distance Transform w/o Background')

ax[5].imshow(watershed_images[index, ..., 0], cmap='jet')
ax[5].set_title('Watershed Segmentation')

fig.tight_layout()
plt.show()
plt.savefig('watershed_organoid_2d')

Image number: 2


# Try with 2D

In [ ]:
filename = 'combined_is_nuclei.npz'
DATA_FILE = os.path.join(DATA_DIR, filename)

print("Loading data from " + filename)
train_dict, test_dict = get_data(DATA_FILE, mode='conv', test_size=0.1, seed=0)

train_dict['X'] = train_dict['X'].reshape((8*24, 160, 160, 1))
train_dict['y'] = train_dict['y'].reshape((8*24, 160, 160, 1))

test_dict['X'] = test_dict['X'].reshape((24, 160, 160, 1))
test_dict['y'] = test_dict['y'].reshape((24, 160, 160, 1))

print(' -\nX.shape: {}\ny.shape: {}'.format(test_dict['X'].shape, test_dict['y'].shape))

In [26]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

fgbg_model_name = 'conv_fgbg_model'
conv_model_name = 'conv_watershed_model'

n_epoch = 5  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 0  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

# Transformation settings
transform = 'watershed'
distance_bins = 4
erosion_width = 0  # erode edges

In [27]:
from deepcell import model_zoo
from deepcell.training import train_model_conv


fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=tuple(train_dict['X'].shape[1:]),
    last_only=False)

fgbg_model = train_model_conv(
    model=fgbg_model,
    train_dict=train_dict,
    test_dict=test_dict,
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    n_epoch=n_epoch,
    batch_size=batch_size,
    transform='fgbg',
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2))

In [30]:
from deepcell import model_zoo
from deepcell.training import train_model_conv


watershed_model = model_zoo.bn_feature_net_skip_2D(
    fgbg_model=fgbg_model,
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_features=distance_bins,
    n_conv_filters=32,
    n_dense_filters=128,
    last_only=False,
    input_shape=tuple(train_dict['X'].shape[1:]))

watershed_model = train_model_conv(
    model=watershed_model,
    train_dict=train_dict,
    test_dict=test_dict,
    model_name=conv_model_name,
    test_size=test_size,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=n_epoch,
    transform=transform,
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2))

In [43]:
# make predictions on testing data
test_images = watershed_model.predict(test_dict['X'])
test_images_fgbg = fgbg_model.predict(test_dict['X'])

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

# Collapse predictions into semantic segmentation mask

argmax_images = []
for i in range(test_images.shape[0]):
    max_image = np.argmax(test_images[i], axis=-1)
    argmax_images.append(max_image)
argmax_images = np.array(argmax_images)
argmax_images = np.expand_dims(argmax_images, axis=-1)

print('watershed argmax shape:', argmax_images.shape)

# threshold the foreground/background
# and remove background from watershed transform
threshold = 0.5
fg_thresh = test_images_fgbg[..., 1] > threshold

fg_thresh = np.expand_dims(fg_thresh.astype('int16'), axis=-1)
argmax_images_post_fgbg = argmax_images * fg_thresh

watershed argmax shape: (24, 160, 160, 1)


In [44]:
# Apply watershed method with the distance transform as seed
from skimage.measure import label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

watershed_images = []
for i in range(argmax_images_post_fgbg.shape[0]):
    image = fg_thresh[i, ..., 0]
    distance = argmax_images_post_fgbg[i, ..., 0]

    local_maxi = peak_local_max(test_images[i, ..., -1], min_distance=15, 
                                exclude_border=False, indices=False, labels=image)

    markers = label(local_maxi)
    segments = watershed(-distance, markers, mask=image)
    watershed_images.append(segments)

watershed_images = np.array(watershed_images)
watershed_images = np.expand_dims(watershed_images, axis=-1)

In [47]:
import matplotlib.pyplot as plt

index = np.random.randint(low=0, high=test_dict['X'].shape[0])
print('Image number:', index)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(test_dict['X'][index, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, ..., 1])
ax[1].set_title('Segmentation Prediction')

ax[2].imshow(fg_thresh[index, ..., 0], cmap='jet')
ax[2].set_title('FGBG Threshold {}%'.format(threshold * 100))

ax[3].imshow(argmax_images[index, ..., 0], cmap='jet')
ax[3].set_title('Distance Transform')

ax[4].imshow(argmax_images_post_fgbg[index, ..., 0], cmap='jet')
ax[4].set_title('Distance Transform w/o Background')

ax[5].imshow(watershed_images[index, ..., 0], cmap='jet')
ax[5].set_title('Watershed Segmentation')

fig.tight_layout()
plt.show()
plt.savefig('watershed_nuclei_2d')

Image number: 14
